In [1]:
from pyspark.sql.functions import when, col, row_number
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType, StringType
import requests
import time
from pyspark.sql.functions import when, col

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 3, Finished, Available)

Bronze - Silver: SCD Type 2.

Those table below is not one of them. Just skip and load it to Silver

## **Customer Region**

In [2]:
# old table and new loading table
dim_store = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_store")
print(dim_store.count())
customer_region = spark.sql("SELECT * FROM LTT_SilverLakehouse.customer_region")
print(customer_region.count())

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 4, Finished, Available)

72
1646


In [3]:
dim_store.createOrReplaceTempView('dim_store')
customer_region.createOrReplaceTempView('customer_region')
dim_store.show(5)
customer_region.show(5)

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 5, Finished, Available)

+---------+----------------------+-------------+-----------+----------------+------------------+------------+-----------+
|store_key|concat_customer_region|store_country|store_state|      store_city|        store_name|   longitude|   latitude|
+---------+----------------------+-------------+-----------+----------------+------------------+------------+-----------+
|        1|  EE. UU.AZPhoenix1...|      EE. UU.|         AZ|         Phoenix|Rocky Shadow Ledge|-112.1697769|33.47337341|
|        2|  EE. UU.CAFremont1...|      EE. UU.|         CA|         Fremont|        Noble Bend| -121.920578|37.33236313|
|        3|  EE. UU.CAFullerto...|      EE. UU.|         CA|       Fullerton|      Umber Meadow|-118.0005798|33.87593842|
|        4|  EE. UU.CAHuntingt...|      EE. UU.|         CA|Huntington Beach|     Quaking Pines|-117.9904709|33.70994186|
|        5|  EE. UU.CALos Ange...|      EE. UU.|         CA|     Los Angeles|       Little Walk|-118.3382416|33.98547363|
+---------+-------------

In [4]:
# FILTER DU LIEU MOI KHONG NAM TRONG DU LIEU CU 
customer_region = \
spark.sql("""
    SELECT *
    FROM customer_region
    WHERE concat_customer_region
    NOT IN (
        SELECT concat_customer_region
        FROM dim_store
    )
""")
customer_region.count()

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 6, Finished, Available)

1574

In [5]:
customer_region = customer_region.withColumnRenamed('customer_country', 'store_country') \
                                .withColumnRenamed('customer_state', 'store_state') \
                                .withColumnRenamed('customer_city', 'store_city') \
                                .withColumnRenamed('customer_street', 'store_name')

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 7, Finished, Available)

In [6]:
max_store_key = dim_store.agg(F.max('store_key')).collect()[0][0]
max_store_key

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 8, Finished, Available)

72

In [7]:
window = Window.orderBy('concat_customer_region')
customer_region = customer_region.withColumn("store_key", max_store_key + row_number().over(window))
customer_region = customer_region.select('store_key', 'concat_customer_region', 'store_country', 'store_state', 'store_city', 'store_name', 'longitude', 'latitude')

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 9, Finished, Available)

In [8]:
customer_region.show(5)

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 10, Finished, Available)

+---------+----------------------+-------------+-----------+----------+------------------+------------+-----------+
|store_key|concat_customer_region|store_country|store_state|store_city|        store_name|   longitude|   latitude|
+---------+----------------------+-------------+-----------+----------+------------------+------------+-----------+
|       73|  EE. UU.ARConway77...|      EE. UU.|         AR|    Conway|       Quiet Mall |-92.43454742|35.09189606|
|       74|  EE. UU.ARJonesbor...|      EE. UU.|         AR| Jonesboro|Middle Gate Towers|-90.65527344|35.85791779|
|       75|  EE. UU.ARJonesbor...|      EE. UU.|         AR| Jonesboro|       Dewy Villas|-97.45480347|25.91904068|
|       76|  EE. UU.AZChandler...|      EE. UU.|         AZ|  Chandler|Hidden Anchor Port|-111.9437943|33.30524445|
|       77|  EE. UU.AZGlendale...|      EE. UU.|         AZ|  Glendale|       Foggy Glade|-112.1931229|33.66019821|
+---------+----------------------+-------------+-----------+----------+-

## **Destination Order**

In [9]:
dim_destination = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_destination")
desti_order = spark.sql("SELECT * FROM LTT_SilverLakehouse.desti_order")
dim_destination.createOrReplaceTempView('dim_destination')
desti_order.createOrReplaceTempView('desti_order')
dim_destination.show(5)
desti_order.show(5)

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 11, Finished, Available)

+---------+--------------------------+------------+------------+-------------+---------------+------------+------------------+-----------+
|desti_key|concat_destination_address|  desti_city| desti_state|desti_country|   desti_region|desti_market|         longitude|   latitude|
+---------+--------------------------+------------+------------+-------------+---------------+------------+------------------+-----------+
|        1|      ApopaSan Salvador...|       Apopa|San Salvador|  El Salvador|Central America|       LATAM|       -89.1790784| 13.8013036|
|        2|      BayamoGranmaCubaC...|      Bayamo|      Granma|         Cuba|      Caribbean|       LATAM|       -76.6492015| 20.3734356|
|        3|      Buenos AiresBueno...|Buenos Aires|Buenos Aires|    Argentina|  South America|       LATAM|         -58.38153|-34.6037181|
|        4|      CarrefourOuestHai...|   Carrefour|       Ouest|        Haití|      Caribbean|       LATAM|       -72.4092495| 18.5345373|
|        5|      Chinandega

In [10]:
print(dim_destination.count())
print(desti_order.count())

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 12, Finished, Available)

56
312


In [11]:
# FILTER DU LIEU MOI KHONG NAM TRONG DU LIEU CU 
desti_order = \
spark.sql("""
    SELECT *
    FROM desti_order
    WHERE concat_destination_address
    NOT IN (
        SELECT concat_destination_address
        FROM dim_destination
    )
""")
desti_order.count()

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 13, Finished, Available)

256

In [12]:
def get_lat_long(city, country, retries=3, delay=2):
    address = f"{city}, {country}"
    url = f"https://nominatim.openstreetmap.org/search?q={address}&format=json&user-agent=MyApplication"
    
    for attempt in range(retries):
        response = requests.get(url, headers={'User-Agent': 'MyApplication'})
        
        if response.status_code == 200:
            data = response.json()
            if data:  # Check if data is not empty
                latitude = float(data[0]["lat"])
                longitude = float(data[0]["lon"])
                return latitude, longitude
            else:
                return None, None  # No results found
        elif response.status_code == 403:
            print(f"403 Forbidden error encountered. Attempt {attempt + 1} of {retries}. Retrying in {delay} seconds...")
            time.sleep(delay)
        else:
            print(f"Error: {response.status_code}")
            return None, None
    
    print("Failed to retrieve coordinates after several attempts.")
    return None, None

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 14, Finished, Available)

In [13]:
# Define UDF to get latitude
def get_lat(city, country):
    lat, lon = get_lat_long(city, country)
    return lat

# Define UDF to get longitude
def get_lon(city, country):
    lat, lon = get_lat_long(city, country)
    return lon

# Register the UDFs
get_lat_udf = udf(get_lat, DoubleType())
get_lon_udf = udf(get_lon, DoubleType())

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 15, Finished, Available)

In [14]:
desti_order = desti_order \
    .withColumn('latitude', get_lat_udf(desti_order['order_city'], desti_order['order_country'])) \
    .withColumn('longitude', get_lon_udf(desti_order['order_city'], desti_order['order_country']))

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 16, Finished, Available)

In [15]:
desti_order = desti_order.withColumnRenamed('order_city', 'desti_city') \
                        .withColumnRenamed('order_state', 'desti_state') \
                        .withColumnRenamed('order_country', 'desti_country') \
                        .withColumnRenamed('order_region', 'desti_region') \
                        .withColumnRenamed('market', 'desti_market')

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 17, Finished, Available)

In [16]:
max_key = dim_destination.agg(F.max('desti_key')).collect()[0][0]
max_key

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 18, Finished, Available)

56

In [17]:
window = Window.orderBy('concat_destination_address')
desti_order = desti_order.withColumn("desti_key", max_key + row_number().over(window))
desti_order = desti_order.select('desti_key', 'concat_destination_address', 'desti_city', 'desti_state', 'desti_country', 'desti_region', 'desti_market')
# because of city and country, can not show the desti_order

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 19, Finished, Available)

## **Load To Silver Table**

In [18]:
print("Number of records need to append")
print(f"Store Count: {customer_region.count()}")
print(f"desti_order Count: {desti_order.count()}")

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 20, Finished, Available)

Number of records need to append
Store Count: 1574
desti_order Count: 256


In [19]:
customer_region.write.format('delta').mode('append').saveAsTable('LTT_GoldLakehouse.dim_store')
desti_order.write.format('delta').mode('append').saveAsTable('LTT_GoldLakehouse.dim_destination')

StatementMeta(, 20d40bea-630b-477d-91e3-64e4bac7280d, 21, Finished, Available)